Automating the @Geezus Metric 
The qb performance from the best college defense they played in their senior year

In [61]:
import cfbd as cfbd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [39]:
cfb_key = open('CFB\\CFBD_key.txt',"r").readline()


In [46]:
from __future__ import print_function
import time
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = cfb_key
configuration.api_key_prefix['Authorization'] = 'Bearer'

# create an instance of the API class
api_instance = cfbd.PlayersApi(cfbd.ApiClient(configuration))
game_id = 56 # int | Game id filter (optional)
year = 2015 # int | Year/season filter for games (optional)
week = 8 # int | Week filter (optional)
season_type = 'regular' # str | Season type filter (regular or postseason) (optional) (default to regular)
team = 'team_example' # str | Team (optional)
home = 'home_example' # str | Home team filter (optional)
away = 'away_example' # str | Away team filter (optional)
conference = 'conference_example' # str | Conference abbreviation filter (optional)


Load in single player

find player on roster

In [49]:
cfb_df = api_instance.player_search('Will Levis',position = 'QB',year = 2022)

In [56]:
curr_team =  cfb_df[0].team



In [59]:
team_api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))


In [66]:
curr_team_list = team_api_instance.get_games(year = 2023, team = curr_team)

In [68]:
pd.DataFrame(curr_team_list)

,0
0,"{'attendance': 58286,\n 'away_conference': 'Mi..."
1,"{'attendance': 61876,\n 'away_conference': 'UA..."
2,"{'attendance': 59456,\n 'away_conference': 'Mi..."
3,"{'attendance': 26279,\n 'away_conference': 'SE..."
4,"{'attendance': 61699,\n 'away_conference': 'SE..."
5,"{'attendance': 92746,\n 'away_conference': 'SE..."
6,"{'attendance': 61654,\n 'away_conference': 'SE..."
7,"{'attendance': 61665,\n 'away_conference': 'SE..."
8,"{'attendance': 52329,\n 'away_conference': 'SE..."
9,"{'attendance': 61936,\n 'away_conference': 'SE..."


In [25]:
cfb_df[(cfb_df['first_name'] == 'Will') & (cfb_df['last_name'] == 'Levis') ]

,athlete_id,first_name,last_name,team,weight,height,jersey,year,position,home_city,home_state,home_country,home_latitude,home_longitude,home_county_fips,recruit_ids,headshot_url,season
5073,4361418,Will,Levis,Penn State,222.0,75.0,NaN,3.0,QB,Madison,CT,USA,41.2794282,-72.5983151,09009,[43559],https://a.espncdn.com/i/headshots/college-foot...,2020


In [14]:
cfb_df.columns

Index(['athlete_id', 'first_name', 'last_name', 'team', 'weight', 'height',
       'jersey', 'year', 'position', 'home_city', 'home_state', 'home_country',
       'home_latitude', 'home_longitude', 'home_county_fips', 'recruit_ids',
       'headshot_url', 'season'],
      dtype='object')